# SA, dwave-neal

- [SA](https://qiita.com/ShataKurashi/items/c0c6044e97fa9e4a9471#:~:text=%E7%84%BC%E3%81%8D%E3%81%AA%E3%81%BE%E3%81%97%E6%B3%95%EF%BC%88%E3%82%84%E3%81%8D%E3%81%AA%E3%81%BE%E3%81%97,%E3%81%A6%E3%80%81%E3%82%88%E3%81%84%E8%BF%91%E4%BC%BC%E3%82%92%E4%B8%8E%E3%81%88%E3%82%8B%E3%80%82)
- [組み合わせ最適問題に対するSA法　中野秀男・中西義郎](http://www.orsj.or.jp/~archive/pdf/bul/Vol.31_01_043.pdf)

In [27]:
import sys
print(sys.path)

['/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython']


In [28]:
import pandas as pd 
import numpy as np
from scipy.special import comb
import math
from neal import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo
import itertools
import random

### 解析用データ作成

（3年間骨肉腫に罹患しない）事象と各特徴量とのP値を計算する。
- Osteogenic sarcoma data:骨肉腫
- LI:リンパ球浸出
- AOP:類骨病理学

In [29]:
ost = pd.read_csv("../input/ost.csv", sep=',', index_col=0)
ost
#論文と照らし合わせ済み

,LI,SEX,AOP,COUNT
Y,,,,
0,0,0,0,0
0,0,0,1,0
0,0,1,0,0
0,0,1,1,0
0,1,0,0,0
0,1,0,1,2
0,1,1,0,4
0,1,1,1,11
1,0,0,0,3


In [30]:
LI_count = 0
for i in range(ost.shape[0]):
    LI_count += ost.iloc[i, 1]*ost.iloc[i, -1]
LI_count    

31

In [31]:
ost46 = pd.DataFrame(np.arange(1*4).reshape(1, 4), columns=['Y', 'LI', 'SEX', 'AOP'])
ost_ = ost.reset_index() #index解除
for i in range(ost_.shape[0]):
    if ost_.iloc[i, -1] > 0:
        for ii in range(ost_.iloc[i, -1]):
            ost46.loc[ost46.shape[0]] = ost_.iloc[i, :-1]

In [32]:
ost46 = ost46.drop(0)

In [33]:
ost46.head()

,Y,LI,SEX,AOP
1,0,1,0,1
2,0,1,0,1
3,0,1,1,0
4,0,1,1,0
5,0,1,1,0


In [34]:
ost46 = ost46.reset_index(drop=True)

In [35]:
ost46.head()

,Y,LI,SEX,AOP
0,0,1,0,1
1,0,1,0,1
2,0,1,1,0
3,0,1,1,0
4,0,1,1,0


In [36]:
ost46.to_csv('../output/ost46.csv')

In [37]:
ost46.to_csv('../4ti2/trial/ost46.csv')

<img src = '../input/about_SA.png'>

In [38]:
t_list = []
count_y = sum([ost.index[i]*ost.iat[i, 3] for i in range(ost.shape[0])])#単に、3年後に骨肉腫に罹患していないサンプル数
t_list.append(count_y)
for j in range(3):
    count = sum([ost.iat[i, j]*ost.iat[i, 3] for i in range(ost.shape[0]) if ost.index[i]==1])
    t_list.append(count)

In [39]:
t_list#Y, LI, SEX, AOP

[29, 19, 16, 12]

### SA

In [40]:
def each_energy(y_sample, t1): #毎回のmaskとyが引数、合致度合いの計算を行う
    energy_except_LI = (np.dot(ost46['SEX'], y_sample)-16)**2 + (np.dot(ost46['AOP'], y_sample) - 12)**2 + (sum(y_sample) - 29)**2
    energy_LI = (np.dot(ost46['LI'], y_sample) - t1)**2
    return energy_except_LI + energy_LI

In [41]:
y_list_num = {}
y_list_variation = {}
each_energy_list = []
t_init = 1e4 #10000
y_sample = ost46['Y'].copy()

for t1 in range(19, 30):
    y_list_num[t1] = 0
    y_list_variation[t1] = []
    for t in reversed(range(1, int(t_init))):
        i = random.randrange(ost46.shape[0])
        y_sample[i] = 1 - y_sample[i]
        if each_energy(y_sample, t1) == 0:
            y_list_num[t1] += 1
            y_list_variation[t1].append(y_sample)
            each_energy_list.append(0)
        elif len(each_energy_list) > 0:
            if np.random.rand() >= np.exp((each_energy_list[-1]-each_energy(y_sample, t1))/t):
                y_sample[i] = 1 - y_sample[i]

In [42]:
y_list_num

{19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0}

In [43]:
columns = list(ost46.columns)
num_samples = ost46.shape[0]

In [ ]:
valid_y_num = {}
for t1 in range(19, 30):
    y = Array.create('y', shape=ost46.shape[0], vartype='BINARY')

    #QUBO式で定式化
    H = (sum(y) - t_list[0])**2
    H += (np.dot(ost46[columns[1]], y) - t1)**2
    for j in range(2, len(columns)):
        H_plus = (np.dot(ost46[columns[j]], y) - t_list[j])**2
        H += H_plus
        
    qb = H.compile().to_qubo()
    sa_sampler = SimulatedAnnealingSampler()
    res = sa_sampler.sample_qubo(qb[0], num_reads=100)

    valid_y_num[t1] = 0
    for y_info in list(res.record):
        if sum(y_info[0]) == t_list[0]:#int同士の比較
            y = pd.Series(y_info[0])
            if np.dot(ost46[columns[1]], y) == t1:#int同士の比較
                for j in range(2, len(columns)):
                    if np.dot(ost46[columns[j]], y) != t_list[j]:#int同士の比較
                        break
                else:
                    valid_y_num[t1] += 1

In [ ]:
valid_y_num

In [ ]:
valid_y_num[19]/sum(valid_y_num.values())